In [1]:
from imu.snowflake import SnowflakeClient, SnowflakeConfig
from ibis import literal
import pyarrow.parquet as pq
import fsspec


import pandas as pd

In [2]:
# All the snow flake interactions use a package called IBIS, which converts pythonis queries in SQL.  
client = SnowflakeClient()
client

SnowflakeClient(connection_info={'user': 'IMU_DAGSTER_PROD_READ_ONLY', 'account': 'RP68682.eu-west-1', 'role': 'IMU_DAGSTER_PROD_READ_ONLY', 'warehouse': 'IMU_WAREHOUSE'})

In [3]:
## SAMPLE TRACKING SHEET ##
samples_table = client.table('V_FCS_TRACKED_SAMPLES', database='IMU.IMU')
#samples_table.limit(5).execute()

In [4]:
## Natural Gates created from Manual Analysis ##
natural_manual = client.table('V_WSP_MEMBERSHIPS_LATEST', database='IMU.IMU_DAGSTER_LINEAGE')
#natural_manual.limit(5).execute()

In [5]:
# Merged together the sample table and manual_natural on the S3 location of the fcs file
natural_manual_metadata = samples_table.join(natural_manual, (literal('s3://') + samples_table['S3 file path']) == natural_manual['FCS_S3_URI'])

In [6]:
# Create a pandas DataFrame 
natural_manual_metadata = natural_manual_metadata.execute() 

In [7]:
# 'MFI_S3_URI' contains that MFI average for each gate
pd.read_csv(natural_manual_metadata.loc[0,'MFI_S3_URI']).head(3)

,marker,root/TIME,root/TIME/CELLS,root/TIME/CELLS/CELLS2,root/TIME/CELLS/CELLS2/FSINGLE,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi/SSChi2,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi/SSChi2/EOSINOPHIL_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi/SSChi2/EOSINOPHIL_1/CD5p_1,...,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1n_TIGITn_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1n_TIGITp_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_KLRG1n_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_KLRG1p_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_TIGITn_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_TIGITp_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/TIM3p_1,"root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/Q1: TCRVA24_JA18 : PE-Dazzle594 - Area- , CD56 : RY586 - Area+",/root,S3 file path
0,FSC:Area,0.179690,0.191503,0.191857,0.183296,0.181887,0.236879,0.235928,0.417819,0.472996,...,0.099769,0.107695,0.114649,0.103745,0.104206,0.137013,0.110588,0.196150,0.180013,s3://imu-flowcytometer-ew1-prod/CW_803106/Othe...
1,FSC:Height,0.095068,0.100873,0.100723,0.099584,0.099576,0.125611,0.125241,0.213801,0.224621,...,0.059914,0.064038,0.069837,0.062379,0.062822,0.080309,0.061566,0.088714,0.095089,s3://imu-flowcytometer-ew1-prod/CW_803106/Othe...
2,FSC:Width,977.890112,978.688033,978.597669,977.894916,977.667013,978.898269,978.870682,979.919770,982.331737,...,975.857852,975.818654,975.495698,975.795616,975.750445,976.223005,977.279959,984.070703,977.900298,s3://imu-flowcytometer-ew1-prod/CW_803106/Othe...


In [8]:
# 'STD_S3_URI' contains the Standard Deviation for each gate
pd.read_csv(natural_manual_metadata.loc[0,'STD_S3_URI']).head(3)

,marker,root/TIME,root/TIME/CELLS,root/TIME/CELLS/CELLS2,root/TIME/CELLS/CELLS2/FSINGLE,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi/SSChi2,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi/SSChi2/EOSINOPHIL_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi/SSChi2/EOSINOPHIL_1/CD5p_1,...,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1n_TIGITn_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1n_TIGITp_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_KLRG1n_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_KLRG1p_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_TIGITn_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_TIGITp_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/TIM3p_1,"root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/Q1: TCRVA24_JA18 : PE-Dazzle594 - Area- , CD56 : RY586 - Area+",/root,S3 file path
0,FSC:Area,0.118461,0.101226,0.101170,0.083772,0.082784,0.068012,0.066992,0.071320,0.089340,...,0.023183,0.041057,0.027938,0.026523,0.025661,0.056186,0.052351,0.051986,0.125794,s3://imu-flowcytometer-ew1-prod/CW_803106/Othe...
1,FSC:Height,0.047137,0.044701,0.041897,0.041011,0.040899,0.034362,0.033930,0.039563,0.051004,...,0.015448,0.021116,0.018137,0.016547,0.016406,0.030627,0.024568,0.023732,0.047206,s3://imu-flowcytometer-ew1-prod/CW_803106/Othe...
2,FSC:Width,7.001201,6.123521,5.968328,3.615254,3.063328,2.917631,2.873617,2.120917,3.158270,...,1.861456,2.039316,1.787372,1.956380,1.932185,3.199454,3.293097,6.743146,7.118494,s3://imu-flowcytometer-ew1-prod/CW_803106/Othe...


In [9]:
# 'COUNTS_S3_URI' contains the counts 
pd.read_csv(natural_manual_metadata.loc[0,'COUNTS_S3_URI'])

,root/TIME,root/TIME/CELLS,root/TIME/CELLS/CELLS2,root/TIME/CELLS/CELLS2/FSINGLE,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi/SSChi2,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi/SSChi2/EOSINOPHIL_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi/SSChi2/EOSINOPHIL_1/CD5p_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSChi/SSChi2/EOSINOPHIL_1/HLADRp_1,...,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1n_TIGITn_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1n_TIGITp_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_KLRG1n_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_KLRG1p_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_TIGITn_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/PD1p_TIGITp_1,root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/OTHER_T/TCRVA7_2p_T_1/MAIT_1/CD8p_1/TIM3p_1,"root/TIME/CELLS/CELLS2/FSINGLE/WBC_1/SSClow/CD3p/CD3p2/TOTAL_T_1/Q1: TCRVA24_JA18 : PE-Dazzle594 - Area- , CD56 : RY586 - Area+",/root,S3 file path
0,1530388,1390512,1232159,1200252,1180066,660166,654868,27931,752,213,...,3272,38,274,2288,2751,37,16,8,1530493,s3://imu-flowcytometer-ew1-prod/CW_803106/Othe...


In [ ]:
# 'EVENTS_S3_URI' contains the individual event MFIs and Memberships 
# Get the S3 path
events_path = natural_manual_metadata.loc[0, "EVENTS_S3_URI"]

# Step 1: Inspect schema with fsspec + pyarrow
with fsspec.open(events_path, "rb") as f:
    parquet_file = pq.ParquetFile(f)
    all_columns = parquet_file.schema.names

# Step 2: Select only columns containing 'root' 
root_columns = [col for col in all_columns if "root" in col.lower()]

# Step 3: Load only those columns into pandas 
df = pd.read_parquet(events_path, columns=root_columns)

df.head()

In [ ]:
# The 'RATIOS_S3_URI' contains the ratios for each gate 
pd.read_csv(natural_manual_metadata.loc[0,'RATIOS_S3_URI'])

In [ ]:
## Natural Gates created from ML-Predicted Analysis ##
natural_predicted = client.table('V_PREDICTED_MEMBERSHIPS_LATEST', database='IMU.IMU_DAGSTER_LINEAGE')
natural_predicted.limit(5).execute()

In [ ]:
## Synthetic Gates created from Manual Analysis ##
synthetic_manual = client.table('V_WSP_SYNTHETIC_MEMBERSHIPS_MANUAL_LATEST', database='IMU.IMU_DAGSTER_LINEAGE')
synthetic_manual.limit(5).execute()

In [ ]:
## Synthetic Gates created from ML-Predicted Analysis ##
synthetic_predicted = client.table('V_WSP_SYNTHETIC_MEMBERSHIPS_PREDICTED_LATEST',database='IMU.IMU_DAGSTER_LINEAGE')
synthetic_predicted.limit(5).execute()